In [166]:
# -*- coding: utf-8 -*-
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from tensorflow import keras

import getpass, ipywidgets as ipw, os, json, shlex, io, re, tempfile, subprocess,unittest
import pydicom,numpy as np,csv,warnings,pickle,sys,tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from IPython.display import FileLink
from matplotlib import pyplot as plt
from zipfile import ZipFile

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
#tf.logging.set_verbosity(tf.logging.ERROR)

%load_ext autoreload
%autoreload 2

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from juxnat_lib.xnat_utils import *
from autoencoder_classifier import DICOMAutoencoderModel

"""
@author: mmilchenko
"""

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'\n@author: mmilchenko\n'

In [167]:
daem=DICOMAutoencoderModel()
daem.load_model_autoencoder('test/autoencoder.256.20K_tokens.09252023.zip')

extracting test/autoencoder.256.20K_tokens.09252023.zip
loading vocabulary from test/autoencoder.256.20K_tokens.09252023
Vocabulary loaded:  test/autoencoder.256.20K_tokens.09252023
loading mdoel from test/autoencoder.256.20K_tokens.09252023.h5


True

In [139]:
daem.model_autoencoder=daem.get_model(len(daem.vectorizer.get_vocabulary()),base_layer_size=256)

In [78]:
daem.vectorizer

In [79]:
daem.autoencoder_model

In [ ]:
#TODO: 
#1. load spreadsheet with pre-existing nomenclature
#2. collect DICOM's from that nomenclature
#3. create arrays 'descs' from those scans
#4. simultaneously, create array with assigned type
#5. generate classifier model
#6. run testing/classification.


In [18]:
#1. load spreadsheet with pre-existing nomenclature.
import csv
def csv_to_dict(filename):
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        data_dict = {field: [] for field in reader.fieldnames}
        for row in reader:
            for field in reader.fieldnames:
                data_dict[field].append(row[field])
    return data_dict

# Example usage:
filename = '/home/mmilchenko/src/scan_classifier/test/all_scans_hofid.csv'
scans = csv_to_dict(filename)
print(scans.keys())


dict_keys(['xnat_imagescandata_id', 'frames', 'series_description', 'ID', 'type', 'URI', 'subject', 'experiment', 'hof_id'])


In [36]:
print(scans['ID'][:10])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [74]:
sp=ServerParams(server='https://cnda.wustl.edu', user='mmilch', password='abc234BB', project='I3CR')
if sp.connect():
    print('connected')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    32  100    32    0     0    159      0 --:--:-- --:--:-- --:--:--   160


connected


In [ ]:
td='/home/mmilchenko/src/scan_classifier/test/dcm'

xi=XnatIterator(sp); xi.verbosity=0
for i in range(10000,100000): #len(scans['URI')
    try:
        name_rt=scans['subject'][i]+'_'+scans['experiment'][i]+'_'+scans['ID'][i]
        #h=tempfile.NamedTemporaryFile(); temp=h.name; h.close()    
        print('listing files for',name_rt)
        files=xi.list_scan_files(scans['subject'][i],scans['experiment'][i],scans['ID'][i])    
        print('downloading first file')
        ff=td+'/'+name_rt+'.full.dcm'    
        xi.curl_download_single_file(files[0],ff)    
        
        #targ_filename='{}/{:06d}.dcm'.format(td,i)
        #xi.curl_download_single_file(scans['URI'][i]+,targ_filename)
        
        print('reading',ff)
        ds=pydicom.filereader.dcmread(ff,stop_before_pixels=True,specific_tags=daem._all_tags)
        ft='{}/{:06d}_{}.dcm'.format(td,i,name_rt)
        print('writing',ft)
        pydicom.filewriter.dcmwrite(ft,ds)
        os.remove(ff)
    except Exception as e:
        print('WARNING: file 0 is missing for scan:',name_rt)
        print(e)


In [80]:
#3.Create the array 'descs' from those scans.
scans_list=daem.generate_scanlist(td,out_file='/home/mmilchenko/src/scan_classifier/test/all_scans_hofid')


reading file 0
reading file 10000
reading file 20000
reading file 30000
writing file /home/mmilchenko/src/scan_classifier/test/all_scans_hofid.pkl.pkl


In [81]:
import os
import re

def reorder_labels(directory_name, label_array):
    # Get list of all files in directory
    files = [f for f in os.listdir(directory_name) if os.path.isfile(os.path.join(directory_name, f))]
    
    # Extract the six-digit numbers from file names using regex
    file_numbers = [int(re.match(r'(\d{6})_', f).group(1)) for f in files if re.match(r'(\d{6})_', f)]
    
    # Sort files based on their numbers
    sorted_file_numbers = sorted(file_numbers)
    
    # Create a new label array using the sorted file numbers
    new_label_array = [label_array[num] for num in sorted_file_numbers]
    
    return new_label_array

scan_types=reorder_labels(td,scans['hof_id'])


In [83]:
scan_types[0]

'OT'

In [53]:
classes=["CBF", "CBV", "DSC", "DWI", "FA", "MD", "MPRAGE", "MTT", "OT", "PBP", "SWI", "T1hi", "T1lo", "T2FLAIR", "T2hi", "T2lo", "TRACEW", "TTP"]

In [85]:
len(daem.vectorizer.get_vocabulary())

20000

In [87]:
#preprocess data.
descs=daem.prepare_descs(scans_list)

In [99]:
descs[0]

'AngioFlag_N BodyPartExamined_BRAIN ImageType0_ORIGINAL ImageType1_PRIMARY ImageType2_FMRI ImageType3_NONE ImageType4_ND ImageType5_MOSAIC InPlanePhaseEncodingDirection_COL MRAcquisitionType_2D Manufacturer_SIEMENS ManufacturerModelName_Skyra Modality_MR PatientPosition_HFS PhotometricInterpretation_MONOCHROME2 ScanOptions0_F ScanOptions1_S ScanningSequence0_E ScanningSequence1_P SequenceName_*epfid2d1_94 SequenceVariant0_S SequenceVariant1_K SeriesDescription_REST1 StudyDescription_I3CR TransmitCoilName_Body VariableFlipAngleFlag_N WindowCenterWidthExplanation0_A WindowCenterWidthExplanation1_l WindowCenterWidthExplanation2_g WindowCenterWidthExplanation3_o WindowCenterWidthExplanation4_1'

In [92]:
def one_hot_encode(labels, all_possible_labels):
    # Initialize the one-hot encoded list
    encoded_list = []

    # For each label in the input labels
    for label in labels:
        # Create a one-hot encoded vector for the label
        encoded_vector = [1 if label == possible_label else 0 for possible_label in all_possible_labels]
        
        # Append the one-hot encoded vector to the list
        encoded_list.append(encoded_vector)

    return encoded_list

In [188]:
n,sum,mx=0,0,0
for desc in descs:
    l=len(desc.split(' '))
    mx=l if l>mx else mx
    sum+=len(desc.split(' '))
    n+=1
print (mx,sum/n)

40 31.177443197372025


In [187]:
len(descs[10000].split(' '))

34

In [93]:
encoded_labels=one_hot_encode(scan_types,["CBF", "CBV", "DSC", "DWI", "FA", "MD", "MPRAGE", "MTT", "OT", "PBP", "SWI", "T1hi", "T1lo", "T2FLAIR", "T2hi", "T2lo", "TRACEW", "TTP"])

In [103]:
def vectorize_text(text,label):
    return daem.vectorizer(text),label

batch_size=64
#create a tf.data.Dataset object, to be used for training.
ds = tf.data.Dataset.from_tensor_slices((descs,encoded_labels))
ds = ds.batch(batch_size)  # Define your batch size here
ds = ds.map(vectorize_text)
dict_size=len(daem.vectorizer.get_vocabulary())

In [104]:
vectorize_text(descs[0],encoded_labels[0])

(<tf.Tensor: shape=(20000,), dtype=float32, numpy=array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)>,
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [168]:
classifier_model=daem.get_classifier_model(daem.autoencoder_model, len(classes), max_tokens=len(daem.vectorizer.get_vocabulary()),hidden_layer_size=512)
classifier_model.summary()

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 encoder_input (Dense)       (None, 256)               5120256   
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 encoder_output (Dense)      (None, 32)                2080      
                                                                 
 dense_36 (Dense)            (None, 128)               4224      
                                                                 
 dense_37 (Dense)            (None, 18)                232

In [169]:
#classifier_model.optimizer.learning_rate.assign(0.0001)
classifier_model.fit(ds,epochs=100, callbacks=[keras.callbacks.ModelCheckpoint(filepath='/home/mmilchenko/temp/tensorboard/training',save_best_only=True,monitor='categorical_accuracy')])

Epoch 1/100
571/571 [==============================] - 5s 7ms/step - loss: 0.5769 - accuracy: 0.8612 - categorical_accuracy: 0.8612
Epoch 2/100
571/571 [==============================] - 3s 5ms/step - loss: 0.2719 - accuracy: 0.9201 - categorical_accuracy: 0.9201
Epoch 3/100
571/571 [==============================] - 3s 5ms/step - loss: 0.2487 - accuracy: 0.9254 - categorical_accuracy: 0.9254
Epoch 4/100
571/571 [==============================] - 3s 6ms/step - loss: 0.2404 - accuracy: 0.9277 - categorical_accuracy: 0.9277
Epoch 5/100
571/571 [==============================] - 3s 5ms/step - loss: 0.2333 - accuracy: 0.9296 - categorical_accuracy: 0.9296
Epoch 6/100
571/571 [==============================] - 4s 7ms/step - loss: 0.2300 - accuracy: 0.9302 - categorical_accuracy: 0.9302
Epoch 7/100
571/571 [==============================] - 3s 6ms/step - loss: 0.2269 - accuracy: 0.9313 - categorical_accuracy: 0.9313
Epoch 8/100
571/571 [==============================] - 3s 5ms/step - loss: 0